<a href="https://colab.research.google.com/github/cnovak232/DL_Speech_Enhancement/blob/main/DL_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torchaudio

from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive
